# Self-Contained Financial Analysis and Insights Notebook

This notebook contains all the necessary code to analyze financial data, categorize transactions, generate insights using AI, and create PDF reports. No external .py files are required.

## License

This notebook is licensed under the GPLv3 License. See the LICENSE file for more details.
- Copyright (C) 2024 [Vincent Koc](https://github.com/vincentkoc)
- Copyright (C) 2024 [Hung Nguyen](https://github.com/hung-ngm)
## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
## Install requsuite packages
%pip install python-dotenv pandas gspread oauth2client openai==0.28.0 pdfkit gradio

In [ ]:
## Make a project directory within colab for this project and MANUALLY upload the .env file to it
import os
from google.colab import files

uploaded = files.upload()  # Prompts you to upload files from your local machine

os.makedirs("/content/ai_personalFinAdvisor", exist_ok=True)
for filename in uploaded.keys():
    os.rename(filename, f"/content/ai_personalFinAdvisor/.env")

os.chdir("/content/ai_personalFinAdvisor")
print(f"Changed Working Directory: {os.getcwd()}")

In [ ]:
## Load .env variables and setup the OpenAI API key
import pandas as pd
import openai
from dotenv import load_dotenv
import pdfkit

# Load environment variables
load_dotenv()

# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is set
if not openai.api_key:
    raise ValueError("OpenAI API key is not set. Please check your .env file.")

## Data Extraction

Let's define functions to load our financial data from either a CSV file or Google Sheets.

In [ ]:
## Make a project 'data' sub-directory within colab for this project and MANUALLY upload the data file to it
from google.colab import files

uploaded = files.upload()  # Prompts you to upload files from your local machine

os.makedirs("/content/ai_personalFinAdvisor/data", exist_ok=True)
for filename in uploaded.keys():
    os.rename(filename, f"/content/ai_personalFinAdvisor/data/nab_cRCard_2023-24.csv")

os.chdir("/content/ai_personalFinAdvisor")
print(f"Changed Working Directory: {os.getcwd()}")

In [ ]:
## Load and examine the financial data
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def load_from_csv():
    csv_path = './data/nab_cRCard_2023-24.csv'  # Adjust path as needed
    absolute_path = os.path.abspath(csv_path)  # Get the absolute path (for debugging)

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"The file {absolute_path} does not exist.")

    return pd.read_csv(csv_path)

# # Load the financial data
df = load_from_csv()

# # Display the first few rows of the dataframe
print(df.head())

# # Display basic information about the dataframe
print(df.info())

In [ ]:
## Clean transaction data
df = df.rename(columns={"Category": "category_existing"})
df['category_existing'] = df['category_existing'].replace(["CafÃ© & coffee", "Café & coffee"], "Cafe & coffee")
df['category_enhanced'] = pd.NA  # Creates an empty column with null values

In [ ]:
# # Display the first few rows of the dataframe
print(df.head())

## Transaction Categorization

Now, let's define functions to categorize the transactions using the OpenAI API.

In [ ]:
##

def categorize_transaction(description):
    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial assistant. Categorize transactions into one of the following categories: Medical, Insurance, Groceries, Restaurants & takeaway, CafÃ© & coffee, Other shopping, Subscriptions, Parking & tolls, Media, Uncategorised, Fuel, Alcohol, Vehicle expenses, Personal care, Education, Public transport, Electronics & technology, Home improvements, Internal transfers, Phone & internet, Donations, Clothing & accessories, Attractions & events, Hobbies, Utilities, Pets, Government, Refund, Travel expenses, Homeware, Services, Taxis & ride shares, Gym & fitness, Fees, Accommodation, Tobacco, Gifts and Flights."},
            # Groceries, Utilities, Rent, Entertainment, Transportation, Dining Out, Miscellaneous, Health & Fitness, Housing, Investments, Insurance, Charity, and Income."},
            {"role": "user", "content": f"Categorize the following transaction: '{description}'. Respond with the category name only."}
        ]
    )
    return completion.choices[0].message.content.strip()

def categorize_transactions(df):
    for index, row in df[df['category_existing'].isin(['Uncategorised', 'Other shopping'])].iterrows(): # df[df['category_enhanced'].isna()] /// # df[df['Category'] == '']
        transaction_text = f"{row['Transaction Details']} | {row['Merchant Name']}" if pd.notna(row['Merchant Name']) else row['Transaction Details']
        category = categorize_transaction(transaction_text)
        # category = categorize_transaction(row['Transaction Details'])
        df.at[index, 'category_enhanced'] = category # Category
    return df

# Categorize the transactions
df = categorize_transactions(df)

# Display the first few rows of the categorized dataframe
print(df.head(18))

# Show the unique categories
print("Unique categories:")
print(df['category_enhanced'].unique()) # 'Category'

In [ ]:
print(df.head(18))

In [ ]:
csv_filename = "output_transactionsCategorised.csv"
df.to_csv(csv_filename, index=False)  # Save CSV without index

from google.colab import files
files.download(csv_filename)


## Financial Summary

Now that we have categorized our transactions, let's generate a financial summary using the OpenAI API.

In [ ]:
def generate_financial_summary(df):
    # Filter only expenses (negative amounts)
    expense_df = df[df['Amount'] < 0]

    # Calculate total spent
    total_spent = expense_df['Amount'].sum()

    # Group expenses by category
    category_spending = expense_df.groupby('category_existing')['Amount'].sum()

    # Generate summary using OpenAI GPT
    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial assistant. Generate a concise financial summary based on the provided expense data."},
            {"role": "user", "content": f"Generate a financial summary based on the following data:\n"
                                        f"Total Expenses: ${total_spent:.2f}\n"
                                        f"Spending by category: {category_spending.to_dict()}"}
        ]
    )
    return completion.choices[0].message.content.strip()

# Generate the financial summary
summary = generate_financial_summary(df)

print("Financial Summary:")
print(summary)

## Personalized Financial Advice

Let's generate some personalized financial advice based on the transaction data and some additional personal information.

In [ ]:
def generate_personalized_advice(df, age=None, lifestyle=None, hobbies=None):
    transactions = df.to_string(index=False)

    prompt = f"""
    Based on the following transaction data:

    {transactions}

    And considering the following personal information:
    Age: {age}
    Lifestyle: {lifestyle}
    Hobbies: {hobbies}

    Please provide personalized financial advice. Include suggestions for budgeting, saving, and potential areas for improvement.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful financial advisor."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

# Generate personalized advice
advice = generate_personalized_advice(df, age=55, lifestyle="Urban", hobbies="Surfing, Trekking, Skateboarding")

print("Personalized Financial Advice:")
print(advice)

## PDF Report Generation

Now, let's create a function to generate a PDF report containing the financial summary and personalized advice.

In [ ]:
!apt-get install -y wkhtmltopdf

In [ ]:
!which wkhtmltopdf

In [ ]:
def create_pdf_report(summary, advice, output_path='financial_report.pdf'):
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>Financial Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; line-height: 1.6; color: #333; }}
            h1 {{ color: #2c3e50; text-align: center; }}
            h2 {{ color: #34495e; }}
            h3 {{ color: #2980b9; }}
            .section {{ margin-bottom: 20px; }}
            ul {{ padding-left: 20px; }}
        </style>
    </head>
    <body>
        <h1>Financial Report</h1>

        <div class="section">
            <h2>Financial Summary</h2>
            {0}
        </div>

        <div class="section">
            <h2>Personalized Financial Advice</h2>
            {1}
        </div>
    </body>
    </html>
    """.format(
        summary.replace('**', '').replace('###', '<h3>').replace('\n', '<br>'),
        advice.replace('**', '').replace('###', '<h3>').replace('\n', '<br>')
    )

    options = {
        'page-size': 'A4',
        'margin-top': '0.75in',
        'margin-right': '0.75in',
        'margin-bottom': '0.75in',
        'margin-left': '0.75in',
        'encoding': "UTF-8",
    }

    pdfkit.from_string(html_content, output_path, options=options)
    print(f"PDF report generated: {output_path}")

# Generate the PDF report
create_pdf_report(summary, advice)

print("PDF report has been generated.")

In [ ]:
from google.colab import files

files.download("financial_report.pdf")  # Replace with your actual PDF filename
